# Opening a new vegan restaurant in Chicago

## Introduction/Business Problem

There has been a slow shift in interests to a vegan lifestyle in the Chicago area.  In general the focus on being healthier has become of greater and greater concern.  Even those who do not want to eat vegan 100% of the time have seen a need to eat healthier more often.  The problem though is there seems to be a lack of vegan restaurants in the Chicago area.  The potential to take advantage of this growing trend could prove to be quite profitable.  Therefore to the right stakeholder it would prove beneficial to examine the city of Chicago's neighborhoods to ascertain the best possible locations for success and profitability.  It is then the goal of this project to come up with the top 2 locations it seems likely a new vegan restaurant would succeed in Chicago.

## Data Process

The data acquisition of the data will come from the Foursquare database.  It is currently the most reliant and up to date database for this type of project.  The initial radius will be set to 1500 do to the fact lake Michigan covers a significant part of this radius.  This may begin to include some of the suburbs but by doing so there is a hope it will deliver the best possible solutions for the stakeholders.  In effort to do this the data we will be looking for will be focused on where there are current vegan restaurants since those are areas have already shown a demand for the cuisine.

Once the data has been acquired, the data will be cleaned and analyzed to reveal the best possible solutions based on the following criteria.
    
    1. Vegan restaurants in the area
    2. Ratings of vegan restaurants in each area

## Methodology

The methodology of the project will include the following processes.

    1. Data acquisition from the Foursquare API
    2. Data wrangling and cleaning
    3. Data normalization if necessary
    4. If necessary K-means clustering of the data to ascertain the areas that have vegan restaraunts
    5. Mapping and visualization of the data
    6. Final analysis and conclusions
    

###### Import the necessary libraries for the project

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!pip install geopy

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium

print('Libraries imported.')

     |████████████████████████████████| 79 kB 8.7 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=a54d782d0686e6a2010357539f264d5a7f0b0988c2bed477b9a59b95f89b0a28
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


###### Acquire the geographical coordinate of Chicago

In [2]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="chicago_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Chicago are 41.8755616, -87.6244212.


###### Establish a Foursquare API connection

In [3]:
CLIENT_ID = '143QSD4JTIZGMNAGDNIX3LJXZE1MHGHLCQ2EWATAPWKJOLXS'
CLIENT_SECRET = '1GCWBBMRIWVG205PEDLIOXXT23HITSWNWQUUVI4WAE1WQ45R'
VERSION = '20180605'
LIMIT = 200

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 143QSD4JTIZGMNAGDNIX3LJXZE1MHGHLCQ2EWATAPWKJOLXS
CLIENT_SECRET:1GCWBBMRIWVG205PEDLIOXXT23HITSWNWQUUVI4WAE1WQ45R


###### After playing with different radius values, I found that 13500 seems to be a sweet spot.

In [4]:
search_veg = 'vegan'

radius = 13500
LIMIT = 200

url_restaurant1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_veg, radius, LIMIT)

results_veg = requests.get(url_restaurant1).json()
results_veg

{'meta': {'code': 200, 'requestId': '601afc3b1aab3a6f15c2bd4f'},
 'response': {'venues': [{'id': '56f17def498edf1990c8d3bb',
    'name': 'Vegan Now',
    'location': {'address': '131 N Clinton St',
     'lat': 41.88413656105296,
     'lng': -87.64080590194325,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.88413656105296,
       'lng': -87.64080590194325}],
     'distance': 1659,
     'cc': 'US',
     'city': 'Chicago',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['131 N Clinton St', 'Chicago, IL', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1d3941735',
      'name': 'Vegetarian / Vegan Restaurant',
      'pluralName': 'Vegetarian / Vegan Restaurants',
      'shortName': 'Vegetarian / Vegan',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1612381243',
    'hasPerk': False},
   {'id': '50930a0ce4b0b68d671d699c',
  

###### Coverting the results to a panda dataframe and normalizing the data

In [5]:
ven = results_veg['response']['venues']
results_veg = pd.json_normalize(ven)
results_veg.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.lat', 'location.lng', 'location.labeledLatLngs',
       'location.distance', 'location.cc', 'location.city', 'location.state',
       'location.country', 'location.formattedAddress', 'location.postalCode',
       'location.crossStreet', 'delivery.id', 'delivery.url',
       'delivery.provider.name', 'delivery.provider.icon.prefix',
       'delivery.provider.icon.sizes', 'delivery.provider.icon.name',
       'location.neighborhood', 'venuePage.id'],
      dtype='object')

###### The revealed data shows how few vegan restaurants there are in Chicago and therefore the potential for capitalization in this growing market.

In [6]:
results_veg

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.crossStreet,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood,venuePage.id
0,56f17def498edf1990c8d3bb,Vegan Now,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1612381243,False,131 N Clinton St,41.884137,-87.640806,"[{'label': 'display', 'lat': 41.88413656105296...",1659,US,Chicago,IL,United States,"[131 N Clinton St, Chicago, IL, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50930a0ce4b0b68d671d699c,Damenzo's Vegan,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1612381243,False,NaN,41.884213,-87.633650,"[{'label': 'display', 'lat': 41.88421270313903...",1229,US,Chicago,IL,United States,"[Chicago, IL 60612, United States]",60612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e1a0db0770046000887643e,Sam & Gertie’s Vegan Jewish Deli,"[{'id': '52e81612bcbc57f1066b79fd', 'name': 'J...",v-1612381243,False,1309 W Wilson Ave,41.965337,-87.662928,"[{'label': 'display', 'lat': 41.965337, 'lng':...",10490,US,Chicago,IL,United States,"[1309 W Wilson Ave (Malden St), Chicago, IL 60...",60640,Malden St,2014010,https://www.grubhub.com/restaurant/sam--gertie...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
3,4ebc2641dab41f33bf06abc7,Vegan Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1612381243,False,NaN,41.894837,-87.630541,"[{'label': 'display', 'lat': 41.89483741261595...",2204,US,Chicago,IL,United States,"[Chicago & Franklin, Chicago, IL 60610, United...",60610,Chicago & Franklin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4f6656f0e4b06cb9fcfec16a,The Vegan Palace,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1612381243,False,3000 Wesley Ave,41.837684,-87.790816,"[{'label': 'display', 'lat': 41.837684, 'lng':...",14426,US,Berwyn,IL,United States,"[3000 Wesley Ave, Berwyn, IL 60402, United Sta...",60402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,51176b82e4b0e64d2e596758,Vegan Restaurat,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1612381243,False,NaN,41.870594,-87.674385,"[{'label': 'display', 'lat': 41.8705940246582,...",4178,US,Chicago,IL,United States,"[Chicago, IL 60612, United States]",60612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,56e60e28498e556c8b65bfc0,The Vegan Plate,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1612381243,False,1550 W Fullerton Ave,41.925406,-87.667642,"[{'label': 'display', 'lat': 41.925406, 'lng':...",6603,US,Chicago,IL,United States,"[1550 W Fullerton Ave (Ashland Ave), Chicago, ...",60614,Ashland Ave,314361,https://www.grubhub.com/restaurant/vegan-plate...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,North Side,NaN
7,5e2c5358d57f4000080d95bd,Vegan And Vegetarian,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1612381243,False,2907 N Lincoln Ave,41.934635,-87.661385,"[{'label': 'display', 'lat': 41.93463516235351...",7254,US,Chicago,IL,United States,"[2907 N Lincoln Ave, Chicago, IL 60657, United...",60657,NaN,1943911,https://www.grubhub.com/restaurant/im-vegan--v...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
8,51176d19e4b00304756eb4ad,eat Vegannnnn,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1612381243,False,NaN,41.870595,-87.674383,"[{'label': 'display', 'lat': 41.87059508645022...",4178,US,Chicago,IL,United States,"[Chicago, IL 60612, United States]",60612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4dc5a32aae608779d140d18e,Vegan Gallery,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",v-1612381243,False,2153 W Division St,41.902833,-87.681791,"[{'label': 'display', 'lat': 41.902833, 'lng':...",5640,US,Chicago,IL,United States,"[2153 

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in results_veg.columns if col.startswith('location.')] + ['id']
vegan_filtered = results_veg.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
vegan_filtered['categories'] = vegan_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
vegan_filtered.columns = [column.split('.')[-1] for column in vegan_filtered.columns]

vegan_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,crossStreet,neighborhood,id
0,Vegan Now,Vegetarian / Vegan Restaurant,131 N Clinton St,41.884137,-87.640806,"[{'label': 'display', 'lat': 41.88413656105296...",1659,US,Chicago,IL,United States,"[131 N Clinton St, Chicago, IL, United States]",NaN,NaN,NaN,56f17def498edf1990c8d3bb
1,Damenzo's Vegan,Vegetarian / Vegan Restaurant,NaN,41.884213,-87.633650,"[{'label': 'display', 'lat': 41.88421270313903...",1229,US,Chicago,IL,United States,"[Chicago, IL 60612, United States]",60612,NaN,NaN,50930a0ce4b0b68d671d699c
2,Sam & Gertie’s Vegan Jewish Deli,Jewish Restaurant,1309 W Wilson Ave,41.965337,-87.662928,"[{'label': 'display', 'lat': 41.965337, 'lng':...",10490,US,Chicago,IL,United States,"[1309 W Wilson Ave (Malden St), Chicago, IL 60...",60640,Malden St,NaN,5e1a0db0770046000887643e
3,Vegan Food Truck,Food Truck,NaN,41.894837,-87.630541,"[{'label': 'display', 'lat': 41.89483741261595...",2204,US,Chicago,IL,United States,"[Chicago & Franklin, Chicago, IL 60610, United...",60610,Chicago & Franklin,NaN,4ebc2641dab41f33bf06abc7
4,The Vegan Palace,Food Truck,3000 Wesley Ave,41.837684,-87.790816,"[{'label': 'display', 'lat': 41.837684, 'lng':...",14426,US,Berwyn,IL,United States,"[3000 Wesley Ave, Berwyn, IL 60402, United Sta...",60402,NaN,NaN,4f6656f0e4b06cb9fcfec16a
5,Vegan Restaurat,Vegetarian / Vegan Restaurant,NaN,41.870594,-87.674385,"[{'label': 'display', 'lat': 41.8705940246582,...",4178,US,Chicago,IL,United States,"[Chicago, IL 60612, United States]",60612,NaN,NaN,51176b82e4b0e64d2e596758
6,The Vegan Plate,Vegetarian / Vegan Restaurant,1550 W Fullerton Ave,41.925406,-87.667642,"[{'label': 'display', 'lat': 41.925406, 'lng':...",6603,US,Chicago,IL,United States,"[1550 W Fullerton Ave (Ashland Ave), Chicago, ...",60614,Ashland Ave,North Side,56e60e28498e556c8b65bfc0
7,Vegan And Vegetarian,Vegetarian / Vegan Restaurant,2907 N Lincoln Ave,41.934635,-87.661385,"[{'label': 'display', 'lat': 41.93463516235351...",7254,US,Chicago,IL,United States,"[2907 N Lincoln Ave, Chicago, IL 60657, United...",60657,NaN,NaN,5e2c5358d57f4000080d95bd
8,eat Vegannnnn,Vegetarian / Vegan Restaurant,NaN,41.870595,-87.674383,"[{'label': 'display', 'lat': 41.87059508645022...",4178,US,Chicago,IL,United States,"[Chicago, IL 60612, United States]",60612,NaN,NaN,51176d19e4b00304756eb4ad
9,Vegan Gallery,Art Gallery,2153 W Division St,41.902833,-87.681791,"[{'label': 'display', 'lat': 41.902833, 'lng':...",5640,US,Chicago,IL,United States,"[2153 W Division St, Chicago, IL 60622, United...",60622,NaN,NaN,4dc5a32aae608779d140d18e


###### A little more cleaning to make it more readable

In [8]:
filtered_columns2 = ['name', 'categories', 'lat', 'lng','postalCode']
vegan_filtered2 = vegan_filtered.loc[:, filtered_columns2]
vegan_filtered2

,name,categories,lat,lng,postalCode
0,Vegan Now,Vegetarian / Vegan Restaurant,41.884137,-87.640806,NaN
1,Damenzo's Vegan,Vegetarian / Vegan Restaurant,41.884213,-87.633650,60612
2,Sam & Gertie’s Vegan Jewish Deli,Jewish Restaurant,41.965337,-87.662928,60640
3,Vegan Food Truck,Food Truck,41.894837,-87.630541,60610
4,The Vegan Palace,Food Truck,41.837684,-87.790816,60402
5,Vegan Restaurat,Vegetarian / Vegan Restaurant,41.870594,-87.674385,60612
6,The Vegan Plate,Vegetarian / Vegan Restaurant,41.925406,-87.667642,60614
7,Vegan And Vegetarian,Vegetarian / Vegan Restaurant,41.934635,-87.661385,60657
8,eat Vegannnnn,Vegetarian / Vegan Restaurant,41.870595,-87.674383,60612
9,Vegan Gallery,Art Gallery,41.902833,-87.681791,60622


In [9]:
vegan_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(vegan_filtered2.lat, vegan_filtered2.lng, vegan_filtered2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(vegan_map)

# display map
vegan_map

In [10]:
#Rating of venues
rating_df=[]

for k in range(vegan_filtered.shape[0]):
    venue_id = vegan_filtered['id']
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #print(result)
    try:
        rating=result['response']['venue']['rating']
        rating_df.append(rating)
        
    except:
        rating='No Rating Yet'
        rating = [0]*5
        rating_df.append(rating)

In [11]:
ratings_info = {'Name': vegan_filtered2.name, 'Rating': rating_df,'Categories': vegan_filtered2.categories, 'Lat' : vegan_filtered2.lat, 'Lng' : vegan_filtered2.lng,'PostalCode' :vegan_filtered2.postalCode}
Chi_vegan = pd.DataFrame(ratings_info)
Chi_vegan

,Name,Rating,Categories,Lat,Lng,PostalCode
0,Vegan Now,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.884137,-87.640806,NaN
1,Damenzo's Vegan,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.884213,-87.633650,60612
2,Sam & Gertie’s Vegan Jewish Deli,"[0, 0, 0, 0, 0]",Jewish Restaurant,41.965337,-87.662928,60640
3,Vegan Food Truck,"[0, 0, 0, 0, 0]",Food Truck,41.894837,-87.630541,60610
4,The Vegan Palace,"[0, 0, 0, 0, 0]",Food Truck,41.837684,-87.790816,60402
5,Vegan Restaurat,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.870594,-87.674385,60612
6,The Vegan Plate,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.925406,-87.667642,60614
7,Vegan And Vegetarian,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.934635,-87.661385,60657
8,eat Vegannnnn,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.870595,-87.674383,60612
9,Vegan Gallery,"[0, 0, 0, 0, 0]",Art Gallery,41.902833,-87.681791,60622


In [12]:
bad_data = Chi_vegan[(Chi_vegan['Categories'] == 'Mental Health Office') 
                     | (Chi_vegan['Categories'] == 'Clothing Store') 
                     | (Chi_vegan['Categories'] == 'Art Gallery') 
                     | (Chi_vegan['Categories'] == 'Food Service')
                     | (Chi_vegan['Categories'] == 'Jewish Restaurant')].index

In [13]:
Chi_vegan.drop(bad_data, inplace = True)

In [14]:
Chi_vegan

,Name,Rating,Categories,Lat,Lng,PostalCode
0,Vegan Now,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.884137,-87.640806,NaN
1,Damenzo's Vegan,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.884213,-87.633650,60612
3,Vegan Food Truck,"[0, 0, 0, 0, 0]",Food Truck,41.894837,-87.630541,60610
4,The Vegan Palace,"[0, 0, 0, 0, 0]",Food Truck,41.837684,-87.790816,60402
5,Vegan Restaurat,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.870594,-87.674385,60612
6,The Vegan Plate,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.925406,-87.667642,60614
7,Vegan And Vegetarian,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.934635,-87.661385,60657
8,eat Vegannnnn,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.870595,-87.674383,60612
10,Vegan Potluck Palace,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.930601,-87.703567,NaN
11,Vegan Mac Down,"[0, 0, 0, 0, 0]",Vegetarian / Vegan Restaurant,41.917383,-87.723951,60647


###### There is no data in the ratings column.  Since Foursquare has no data in this particular area we will have to rely on the general observation viewed on the map.
###### In addition since there is clearly very few vegan restaurants there is little to no need to us K-means clustering.  The final map is produced below.

In [15]:
final_vegan_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(Chi_vegan.Lat, Chi_vegan.Lng, Chi_vegan.Categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(final_vegan_map)

# display map
final_vegan_map

## Results

When looking at the final mapped results it is quite obvious to see where vegan restaurants are in Chicago.  Three areas are clear.  They are Uptown, Lincoln Park and downtown Chicago.  Unfortunately it was hoped there would be a lot more data to help make a better determination.  The goal was to identify 2 location for a new restaurant.  It would seem that the areas of either Uptown or Lincoln Park would be ideal as they both have the highest concentration.  Further observations and recommendations will be in the following discussion section.

## Discussion and Conclusion

The results were less than what I set out to find.  There is clearly a lack of vegan restaurants in Chicago.  With the growing demand the areas of Uptown and Lincoln Park are the most concentrated and seem the most ideal.  However, having personally knowing the city a bit there may be other areas to consider.  Finding another source for data would be important.  Unfortunatly I do not know of another source of reliable data for this particular project.  Some things to consider would be demographics or possibly foot traffic.  The heart of the city know as the Loop has a good daytime population but in most of the areas vacates at night as an example.  Discussing this with the stakeholder is therefore important to see if their needs are met within the scope of this project.

## Conclusion

The conclusion of this project that further research may be necessary in this particular case in order to determine the best spot for a new vegan restaurant. Although it is clear that if the stakeholder would like to make a decision based on this project's scope then they would certainly find success in either the Uptown and Lincoln Park areas of the city.